In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
import splat
import astropy.units as u
import popsims
splat.initializeStandards(all=True)
from scipy import interpolate
%matplotlib inline

Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database
Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database






In [2]:
files=glob.glob('/users/caganze/research/wisps/data/primevalspectra/*.fits')

In [3]:
#files

In [4]:
def get_spectrum(f):
        hdu=fits.open(f)[0]
        wave=np.array([hdu.header['CRVAL1']+hdu.header['CDELT1']*x for x \
          in range(0, hdu.header['NAXIS1'])])*u.angstrom.to(u.micron)
        flux= hdu.data*u.erg/(u.cm**2 * u.s * u.micron)
        s=splat.Spectrum(wave=wave, flux=flux, noise=flux/1000)
        #s.toInstrument('SpeX')
        #plt.plot(s.wave, s.flux)
        return s

In [5]:
def interpolated_standards():
    stds=splat.STDS_DWARF_SPEX
    interpstds={}
    for k in stds.keys():
        s=stds[k]
        print (k)
        s.normalize()
        s.toInstrument('WFC3-G141')
        interpstds[k]=(interpolate.interp1d(s.wave.value, s.flux.value), interpolate.interp1d(s.wave.value, s.noise.value))
    return interpstds

In [6]:
roman= pd.read_pickle(popsims.DATA_FOLDER+'/chris.pkl')

In [7]:
evol=pd.DataFrame.from_records(roman).T#()
#age_keys= [x for x in roman.keys()]

In [8]:
evol['age_key']=[x for x in evol.index]

In [9]:
evol=evol.reset_index(drop=True)

In [10]:
evol['age']=evol.apply(lambda x: np.ones(len(x.teff))*x.age_key/1e9, axis=1)

In [11]:
#evol.

In [12]:
final_evol=pd.DataFrame()
for c in evol.drop(columns=['age_key']).columns:
    final_evol[c]= np.concatenate(evol[c])

In [13]:
final_evol['teff']=final_evol.teff.apply(np.round)

In [18]:
final_evol=final_evol.rename(columns={'teff': 'temperature', 
                           'lum': 'luminosity',
                          'logg': 'gravity'})

In [19]:
final_evol.to_csv(popsims.EVOL_MODELS_FOLDER+'/roman2020.csv')

In [20]:
final_evol.max()

luminosity        0.321536
temperature    6343.000000
mass              0.607143
gravity           5.622532
age              13.500000
dtype: float64